### Problem Set 10: Machine Learning

In this problem set, you will implement Naive Bayes, Logistic Regression, and Neural Network training.

0. [Credit for Contributors (required)](#contributors)
1. [Naive Bayes (45 points)](#problem1)
    1. [Count Word Frequency (5 points)](#get_counts)
    2. [Learn Parameters (20 points)](#NB_learn_params)
    3. [Classify Message (10 points)](#NB_classify)
    4. [Observations: Priors and Smoothing (10 points)](#NB_observations)
2. [Logistic Regression (40 points)](#problem2)
    1. [Loss Function (5 points)](#logistic_loss)
    2. [Gradient Descent (30 points)](#gradient_descent)
    3. [Learning Rate (5 points)](#learning_rate)
3. [Neural Networks (30 points)](#problem3)
    1. [Define Neural Network (15 points)](#define_NN)
    2. [Prevent Overfitting (15 points)](#overfitting)
4. [Homework survey (5 points)](#part4)
    
**120 points** total for Problem Set 10

## <a name="contributors"></a> Credit for Contributors

List the various students, lecture notes, or online resouces that helped you complete this problem set:

Ex: I worked with Bob on the cat activity planning problem.

<div class="alert alert-info">
Write your answer in the cell below this one.
</div>

--> *(double click on this cell to delete this text and type your answer here)*

In [ ]:
# Be sure to run the cell below to import the code needed for this assignment.
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification, make_moons
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from collections import defaultdict

from utils import *

# imports for autograder
from principles_of_autonomy.grader import Grader
from principles_of_autonomy.notebook_tests.pset_10 import TestPSet10

## <a name="problem1"></a> Problem 1: Naive Bayes (45 points)

In this problem, you will build a Naive Bayes classifier to detect spam. Follow the steps below to implement the required functions and test your classifier.

We are giving you a dataset of [5,572 SMS messages](https://archive.ics.uci.edu/dataset/228/sms+spam+collection), which we split into a training and a test set for you. Run the following and have a look at some datapoints.

In [ ]:
# Load and clean dataset. The clean step removes punctuation and makes everything lowercase.
labels, messages = load_data('SMS_data')
labels, messages = clean_data(labels, messages)

# Look at the data.
for i in range(5):
    print(f"{labels[i]} : {messages[i]}")

# Split into training and test sets.
labels, messages = shuffle_data(labels, messages)
(training_labels, training_messages), (test_labels, test_messages) = split_data(labels, messages)

For this problem, you should assume that each message $i$ has an unobserved class variable $C^i$ that takes the value "spam" or "ham". Let $\{w_1 ... w_J\}$ be the set of all words used in all messages. The features of each message are binary word occurrences: $Y^i_j$ is a binary random variable that takes on value 1 if $w_j$ occurs in message $i$, and 0 if it does not. You can imagine that words like "free", "win", and so on would be very likely to occur in spam, but not as likely in ham.

The goal is to classify messages using the Naive Bayes framework. The joint probability of class labels and features is given by:

$$
P(C^1=c^1, \ldots, C^N=c^N, Y^1_1=y^1_1, \ldots, Y^N_J=y^N_J) =
\prod_{i=1}^N \left[ P(C^i=c^i) \prod_{j=1}^J P(Y^i_j=y^i_j \mid c^i) \right]
$$

We will assume that:

$$
P(Y^i_j=y^i_j \mid c^i) =
\begin{cases} 
    \text{Bernoulli}(q_j), & \text{if } c^i = \text{spam} \\
    \text{Bernoulli}(p_j), & \text{if } c^i = \text{ham}
\end{cases}
$$

and

$$
P(C^i=c^i) =
\begin{cases}
    s, & \text{if } c^i = \text{spam} \\
    1 - s, & \text{if } c^i = \text{ham}
\end{cases}
$$

To avoid working with multiplying small probabilities together, we’ll work with log probabilities instead. The equation then becomes:
$$
\log P(C^1=c^1, \ldots, C^N=c^N, Y^1_1=y^1_1, \ldots, Y^N_J=y^N_J) =
\sum_{i=1}^N \left[ \log P(C^i=c^i) + \sum_{j=1}^J \log P(Y^i_j=y^i_j \mid c^i) \right]
$$

The maximum likelihood estimate for the parameter $p_j$ is just the fraction of ham messages that contain the word $w_j$, and similarly, the maximum likelihood estimate for $q_j$ is the fraction of spam messages that contain the word $w_j$. The maximum likelihood estimate for $s$ is the fraction of training documents that are spam. **Our goal is to estimate these parameters and build a classification function** that can classify new messages as spam or ham with the parameters we estimate.


### <a name="get_counts"></a> Count Word Frequency (5 points)

The first step to estimating the Naive Bayes parameters is counting the number of messages each word occurs in. Implement a function `get_counts` that takes in the training messages and returns a dictionary whose keys are words, and whose values are the number of messages the key occurred in. Words are all collections of characters separated by whitespace.

<div class="alert alert-info">
Implement the function `get_counts(messages)` below.
</div>

In [ ]:
def get_counts(messages):
    """
    Computes counts for each word that occurs in the messages.

    Inputs
    ------
    messages : a list of sms messages.

    Output
    ------
    A dict whose keys are words, and whose values are the number of files the key occurred in.
    """
    raise NotImplementedError()

In [ ]:
Grader.run_single_test_inline(TestPSet10, "test_01", locals())

Now look at your counts ordered by value. You should see common English words (e.g. prepositions) at the top of the list.

In [ ]:
word_counts = get_counts(training_messages)
sorted_word_counts = sorted(word_counts.items(), key=lambda item: item[1], reverse=True)
sorted_word_counts[:20]

### <a name="NB_learn_params"></a> Parameter Learning (20 points)

You will now implement 2 functions to estimate the Naive Bayes parameters. 

First, implement `get_log_probabilities`, which, given a list of messages, computes the log of the fraction of messages each word occurrs in. To avoid issues with unseen words, the function also takes a smoothing parameter `k`. You should use [**Laplace smoothing**](https://en.wikipedia.org/wiki/Additive_smoothing) to compute the smoothed frequencies of each word. The function should return a dictionary whose keys are words, and whose values are the log of the smoothed estimate of the fraction of messages the key occurred in.


<div class="alert alert-info">
Implement the functions `get_log_probabilities(messages, k)` and `learn_distributions(messages_by_category, k)` below.
</div>

In [ ]:
def get_log_probabilities(messages, k):
    """
    Computes log-frequencies for each word that occurs in the messages.

    Input
    -----
    messages : a list of sms messages.
    k : Laplace smoothing parameter.

    Output
    ------
    log_prob : a **defaultdict** dictionary whose keys are words, and whose values are the log of the smoothed estimate of the fraction of messages the key occurred in.

    Hint
    ------
    1. The `get_counts` helper function you wrote earlier should be helpful here.
    2. To handle missing words, you should use defaultdict from collections (imported at the top of the file) as your dictionary.
       This allows you to set a default value for keys not in the dictionary, which will come in handy later.
    """
    raise NotImplementedError()

In [ ]:
Grader.run_single_test_inline(TestPSet10, "test_02", locals())

Next, implement `learn_distributions`, which learns the parameters for each conditional distribution $P(Y_j|C)$ and prior distribution $P(C)$ in the model. The function takes a list `messages_by_category` which is a two-element list: the first element is a list of spam messages and the second element is a list of ham messages. The function also takes `k` which you should use to call `get_log_probabilities` for obtaining smoothed frequencies. You should return a tuple containing: 1. a list with a smoothed estimate for $\log P(Y_j=w_j\mid C=spam)$ (as a dictionary) and a smoothed estimate for $\log P(Y_j=w_j\mid C=ham)$ (also as a dictionary); 2. a list with estimates for the log-probabilities for each class, i.e. $\log P(C=spam)$ and $\log P(C=ham)$.

In [ ]:
def learn_distributions(messages_by_category, k):
    """
    Input
    -----
    messages_by_category :  A two-element list. The first element is a list of spam messages, 
                            and the second element is a list of ham (non-spam) messages.
    k : Laplace smoothing parameter.

    Output
    ------
    (log_probs_by_category, log_prior_by_category)

    log_probs_by_category : A list whose first element is a smoothed estimate for log P(y=w_j|c=spam) (as a dict,
                            just as in get_log_probabilities above), and whose second element is the same for c=ham.

    log_prior_by_category : A list of estimates for the log-probabilities for each class: [est. for log P(c=spam), est. for log P(c=ham)]
    """
    log_probs_by_category = []
    log_prior_by_category = []
    raise NotImplementedError()
    return (log_probs_by_category, log_prior_by_category)

In [ ]:
Grader.run_single_test_inline(TestPSet10, "test_03", locals())

### <a name="NB_classify"></a> Classification (10 points)

Implement the function `classify_message`, which classifes a new message using the Maximum A-Posteriori, or MAP, rule. The MAP rule is almost the same as the MLE rule except it additionally multiplies the data likelihood by the prior. The function takes in a string message and the probabilities estimated in the previous part, then uses the equation in the problem description to compute the log probabilities for each category. 

<div class="alert alert-info">
Implement the function `classify_message(message, log_probs_by_category, log_prior_by_category)` below.
</div>

<div class="alert alert-warning">
Hint: for classifying a single message, $N$ and $J$ have different values from those in the training equation above.
</div>

In [ ]:
def classify_message(message, log_probs_by_category, log_prior_by_category):
    """
    Classifies a new message as spam or ham using the MAP rule.

    Inputs
    ------
    message : a string representing the new message to classify.
    log_probs_by_category : a list whose first element is a dict of log P(w_j|c=spam), 
                            and second element is a dict of log P(w_j|c=ham).
    log_prior_by_category : a list of log-priors [log P(c=spam), log P(c=ham)].

    Output
    ------
    A string: "spam" or "ham", indicating the classification result.
    """
    raise NotImplementedError()

In [ ]:
"""Test your classification code here."""
Grader.run_single_test_inline(TestPSet10, "test_04", locals())

Now let's evaluate your classification code on the test set. You should expect about 80% accuracy.

In [ ]:
# Extract spam and ham messages from the training data.
spam_messages = [msg for lbl, msg in zip(training_labels, training_messages) if lbl == 'spam']
ham_messages = [msg for lbl, msg in zip(training_labels, training_messages) if lbl == 'ham']

# Set Laplace smoothing parameter.
k = 1

# Training: learn distribution parameters from the training data.
(log_probs_by_category, log_prior_by_category) = learn_distributions([spam_messages, ham_messages], k)

# Here, columns and rows are indexed by 0 = 'spam' and 1 = 'ham'.
# Rows correspond to true label, columns correspond to guessed label.
performance_measures = np.zeros([2,2])

### Classify and measure performance on the test set.
for label, message in zip(test_labels, test_messages):
    predicted_label = classify_message(message, log_probs_by_category, log_prior_by_category)
    true_index = 0 if label == "spam" else 1
    guessed_index = 0 if predicted_label == "spam" else 1
    performance_measures[true_index, guessed_index] += 1
correct = np.diag(performance_measures)
totals = np.sum(performance_measures, axis=1)
accuracy = (correct[0] + correct[1]) / np.sum(performance_measures)
print(f"You correctly classified {correct[0]} out of {totals[0]} spam messages, and {correct[1]} out of {totals[1]} ham messages. Accuracy: {accuracy*100}%")

### <a name="NB_observations"></a> Observations (10 points)

Estimating $s$ from data is heavily reliant on the sizes of the data. In the real world, it's often difficult to find good training examples for ham, since nobody wants to give out their private email for the world to read. As a result, spam datasets often have many more spam examples than ham examples. By fixing $s$ manually, we can adjust how much the algorithm favors catching spam at the expense of falsely flagging a ham message. Try setting $s$ to a few values manually, and briefly explain what happens to your performance as $s$ increases and decreases.

<div class="alert alert-info">
**Discuss your results in the cell below**
</div>

--> *(double click on this cell to delete this text and type your answer here)*



Finally, the Laplace smoothing parameter is critical for handling unseen words during testing by assigning them a small nonzero probability. However, it also imposes a uniform prior, which smoothens the estimated probabilities of observed words. In the previous code block, we estimated the Naive Bayes parameters using $k = 1$. Experiment with a range of $k$ values and identify the value of $k$ that results in the highest accuracy. Briefly explain how $k$ affects the model's performance.

<div class="alert alert-info">
**Discuss your results in the cell below**
</div>

--> *(double click on this cell to delete this text and type your answer here)*



## <a name="problem2"></a> Problem 2: Logistic Regression (40 points)

In this problem, you will implement **Logistic Regression** to classify points in a 2D space into two classes. We have generated a synthetic dataset of 500 samples, each with two features; the data can be summarized as a $500\times2$ matrix `X` representing the input features and a 1D array `y` representing the true labels corresponding to each datapoint. We split the data for you into a training set and test set. Run the following cell to visualize the data.

In [ ]:
# Generate synthetic data.
X, y = make_classification(
    n_samples=500, n_features=2, n_informative=2, n_redundant=0, 
    n_clusters_per_class=1, class_sep=1.5, flip_y=0.1, random_state=42
)

# Visualize the dataset
plt.figure(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', edgecolors='k')
plt.title("Synthetic Dataset for Logistic Regression")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()

# Augment the feature matrix X by adding a bias column of ones.
X = np.hstack([X, np.ones((X.shape[0], 1))])

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### <a name="logistic_loss"></a> Loss Function (5 points)

The goal of logistic regression is to find the optimal weights that minimize the difference between predicted probabilities and true labels. To achieve this, we use gradient descent, an optimization algorithm that iteratively adjusts the model's parameters. Before we dive into implementing gradient descent, you will first implement three functions that we'll make use of in our model parameter fitting. 

First, implement the `sigmoid` function, which takes a real value as input and returns a value between 0 and 1, representing the probability of belonging to a class. This should be one line of code implementing the following: 

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

Next, implement `binary_cross_entropy`, which is the loss function the gradient descent algorithm will optimize. We talked about Binary Cross Entropy in the neural networks lecture, but the same principle applies for logistic regression: given an array of prediction probabilities `y_pred` and the corresponding true class labels `y_true`, BCE measures how far off the model predictions are. This should also be one line of code implementing:

$$
\text{Loss}(y, \hat{y}) = - \frac{1}{N} \sum_{i=1}^N \left( y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right)
$$

Finally, implement `loss_gradient`, which calculates the gradients of the loss function with respect to the model's parameters. The gradient for logistic regression is $\nabla_w \text{Loss} = \frac{1}{N} \sum_{i=1}^N (\hat{y}_i - y_i) \mathbf{x}_i$, where $\mathbf{x}_i$ is the feature vector for the i-th sample. To see how we got this formula, you can try taking the gradient of the Loss equation above w.r.t. the weights by hand, as an optional (but very educational! :) ) exercise.

<div class="alert alert-info">
Implement the functions `sigmoid(z)`, `binary_cross_entropy(y_true, y_pred)`, and loss_gradient(X, y, y_pred) below.
</div>

In [ ]:
# Sigmoid function
def sigmoid(z):
    """
    Compute the sigmoid of z.

    Input:
    - z : A scalar or numpy array of any size.

    Output:
    - The sigmoid of z.
    """
    raise NotImplementedError()

# Binary cross-entropy loss
def binary_cross_entropy(y_true, y_pred):
    """
    Compute the binary cross-entropy loss.

    Inputs:
    - y_true : A numpy array of true binary labels (0 or 1).
    - y_pred : A numpy array of predicted probabilities (0 <= y_pred <= 1).

    Output:
    - The binary cross-entropy loss.
    """
    epsilon = 1e-15  # To avoid log(0)
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    raise NotImplementedError()

def loss_gradient(X, y, y_pred):
    """
    Compute the gradients of the loss with respect to the augmented weights.

    Inputs:
    - X : A numpy array of shape (N, D+1), the augmented feature matrix.
    - y : A numpy array of shape (N,), the binary labels (0 or 1).
    - y_pred : A numpy array of shape (N,), the predicted probabilities.

    Output:
    - dw : A numpy array of shape (D+1,), the gradient of the loss with respect to w.
    """
    raise NotImplementedError()

In [ ]:
"""Test your loss function code here."""
Grader.run_single_test_inline(TestPSet10, "test_05", locals())

### <a name="gradient_descent"></a> Gradient Descent (30 points)

You will now implement the **Gradient Descent** update rule for logistic regression. In class, we discussed that we can estimate the weights using Maximum Likelihood Estimation, and we wrote logistic regression as an optimization problem where we maximize the log-likelihood of the data given the parameters with gradient ascent. In practice, it's more common to convert the problem into a minimization problem (by negating the log-likelihood; sometimes this is called the *nll* in shorthand notation) and run gradient *descent*. In this problem, you will:
1. Implement batch gradient descent (process all data at once in a single update per epoch).
2. Implement stochastic gradient descent (process one data point at a time from the shuffled dataset).
3. Implement mini-batch gradient descent (process small batches of data from the shuffled dataset).

We provide you with the outer training loop, visualization functions, and evaluation inside `train_and_plot`. You should implement the functions `batch_gradient_descent`, `stochastic_gradient_descent`, and `mini_batch_gradient_descent`, each of which execute a single *epoch*, or iteration, or gradient descent. As a reminder, batch gradient descent makes a single update per epoch, stochastic gradient descent makes $N$ updates per epoch, and mini-batch gradient descent makes $N/J$ updates per epoch (where $J$ is the batch size).

<div class="alert alert-info">
Implement the functions `batch_gradient_descent(X, y, w, alpha)`, `stochastic_gradient_descent(X, y, w, alpha)`, `mini_batch_gradient_descent(X, y, w, alpha, batch_size)` below.
</div>

In [ ]:
def train_and_plot(X_train, y_train, X_test, y_test, gd_type, alpha, epochs=100, batch_size=32):
    """
    Train the logistic regression model using the specified gradient descent method and plot the loss curve.

    Inputs:
    - X_train : Training data (N, D+1).
    - y_train : Training labels (N,).
    - X_test : Test data (M, D+1).
    - y_test : Test labels (M,).
    - gd_type : A string, either 'batch', 'stochastic', or 'mini_batch'.
    - alpha : A float, the learning rate.
    - epochs : An integer, the number of training epochs.
    - batch_size : An integer, the size of each mini-batch (used only for 'mini_batch').

    Outputs:
    - None. Prints accuracy and plots the loss curve.
    """
    # Initialize parameters.
    w = np.zeros(X_train.shape[1])
    losses = []

    for epoch in range(epochs):
        if gd_type == "batch":
            w = batch_gradient_descent(X_train, y_train, w, alpha)
        elif gd_type == "stochastic":
            w = stochastic_gradient_descent(X_train, y_train, w, alpha)
        elif gd_type == "mini_batch":
            w = mini_batch_gradient_descent(X_train, y_train, w, alpha, batch_size)
        else:
            raise ValueError("Invalid gd_type. Choose 'batch', 'stochastic', or 'mini_batch'.")

        # Compute loss for the current epoch for plotting purposes.
        y_pred_train = sigmoid(np.dot(X_train, w))
        loss = binary_cross_entropy(y_train, y_pred_train)
        losses.append(loss)

    # Plot the loss curve.
    plt.plot(losses, label=f"{gd_type} GD")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title(f"Loss Curve for {gd_type.capitalize()} GD")
    plt.legend()
    plt.show()

    # Plot decision boundary.
    plt.figure(figsize=(10, 5))
    for label, color in zip([0, 1], ['blue', 'red']):
        plt.scatter(X_train[y_train == label, 0], X_train[y_train == label, 1], 
                    label=f"Class {label}", alpha=0.7, edgecolors='k', c=color)
    x_vals = np.linspace(X_train[:, 0].min(), X_train[:, 0].max(), 100)
    y_vals = -(w[0] / w[1]) * x_vals - (w[2] / w[1])  # Solve for x2
    plt.plot(x_vals, y_vals, label="Decision Boundary", color="black")
    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.title("Decision Boundary and Data")
    plt.legend()
    plt.show()
    
    # Evaluate on test set
    y_pred_test = sigmoid(np.dot(X_test, w)) >= 0.5
    accuracy = np.mean(y_pred_test == y_test)
    print(f"{gd_type.capitalize()} GD Test Accuracy: {accuracy:.2f}")

# Batch Gradient Descent
def batch_gradient_descent(X, y, w, alpha):
    """
    Perform one step of batch gradient descent using loss_gradient.

    Inputs:
    - X : A numpy array of shape (N, D+1), the augmented feature matrix.
    - y : A numpy array of shape (N,), the binary labels (0 or 1).
    - w : A numpy array of shape (D+1,), the augmented weights.
    - alpha : A float, the learning rate.

    Output:
    - Updated weights w.
    """
    raise NotImplementedError()



In [ ]:
"""Test your BATCH gradient descent code here. NOTE: stochastic and mini_batch are not tested"""
Grader.run_single_test_inline(TestPSet10, "test_06", locals())

In [ ]:
def stochastic_gradient_descent(X, y, w, alpha):
    """
    Perform one epoch of stochastic gradient descent using loss_gradient.
    Goes through the entire dataset randomly.

    Inputs:
    - X : A numpy array of shape (N, D+1), the augmented feature matrix.
    - y : A numpy array of shape (N,), the binary labels (0 or 1).
    - w : A numpy array of shape (D+1,), the augmented weights.
    - alpha : A float, the learning rate.

    Output:
    - Updated weights w.
    """
    raise NotImplementedError()

# Mini-Batch Gradient Descent
def mini_batch_gradient_descent(X, y, w, alpha, batch_size):
    """
    Perform one epoch of mini-batch gradient descent using loss_gradient.
    Goes through the entire dataset in random mini-batches.

    Inputs:
    - X : A numpy array of shape (N, D+1), the augmented feature matrix.
    - y : A numpy array of shape (N,), the binary labels (0 or 1).
    - w : A numpy array of shape (D+1,), the augmented weights.
    - alpha : A float, the learning rate.
    - batch_size : An integer, the number of samples in each mini-batch.

    Output:
    - Updated weights w.
    """
    raise NotImplementedError()

#### Let's test your code and visualize the results (20 points):
We will be grading the below plots. **Do not modify the code below**. You should expect accuracies greater than 80\% for all three methods.

In [ ]:
### DO NOT MODIFY
epochs = 100
alpha = 0.01
batch_size = 32

train_and_plot(X_train, y_train, X_test, y_test, gd_type="batch", alpha=alpha, epochs=epochs)
train_and_plot(X_train, y_train, X_test, y_test, gd_type="stochastic", alpha=alpha, epochs=epochs)
train_and_plot(X_train, y_train, X_test, y_test, gd_type="mini_batch", alpha=alpha, epochs=epochs, batch_size=batch_size)
### DO NOT MODIFY

### <a name="learning_rate"></a> Learning Rate (5 points)
Try varying the learning rate parameter `alpha` in the cell **below**, e.g. 0.001, 0.005, 0.01, 0.05, 0.1, 0.5.
- What are the best learning rates for each of the 3 gradient descent variants? What happens when the learning rate is too small? What about too large?
- Which method is more robust to large learning rates?
- Which method converges fastest when the learning rate is optimal?

<div class="alert alert-info">
**Discuss your results below**
</div>

--> *(double click on this cell to delete this text and type your answer here)*



In [ ]:
### MODIFY THESE!
epochs = 100
alpha = 0.01
batch_size = 32

train_and_plot(X_train, y_train, X_test, y_test, gd_type="batch", alpha=alpha, epochs=epochs)
train_and_plot(X_train, y_train, X_test, y_test, gd_type="stochastic", alpha=alpha, epochs=epochs)
train_and_plot(X_train, y_train, X_test, y_test, gd_type="mini_batch", alpha=alpha, epochs=epochs, batch_size=batch_size)

## <a name="problem3"></a> Problem 3: Neural Networks (30 points)

In this problem, you will define a **Neural Network**, train it with a given dataset, and explore overfitting prevention techniques, all with PyTorch. The goal of this problem is to take you through the journey of what training a neural network with modern autodiff packages looks like. Since PyTorch is almost a new language, you won't have to write too much code and when you do, we will provide guidance. You may want to befriend the [PyTorch documentation](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html). 

First, we generate a dataset and split it into a training set, validation set, and test set. The difference between validation and test is that test is used for the final evaluation of the model (like before), and the validation set is meant for picking hyperparameters or early stopping.

In [ ]:
# Generate dataset.
X, y = make_moons(n_samples=1000, noise=0.2, random_state=42)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

Next, we convert this dataset to PyTorch tensors, which are specialized data structures that are very similar to numpy arrays, but with additional features that make them ideal for deep learning tasks (can run on GPUs or other hardware accelerators, but not for this homework).

In [ ]:
# Convert to PyTorch tensors.
X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
X_val, y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long)
X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)

Finally, we create DataLoaders, which will help handle the dataset batching that you had to do manually in mini-batch gradient descent.

In [ ]:
# DataLoader for batching.
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=32)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32)

### <a name="define_NN"></a> Define Neural Network (15 points)

Your task is to define a **3-layer fully connected neural network** for binary classification using PyTorch. The structure of the network is:
1. Input → Hidden Layer 1 → ReLU Activation.
2. Hidden Layer 1 output (after ReLU) → Hidden Layer 2 → ReLU Activation.
3. Hidden Layer 2 output (after ReLU)→ Output Layer.

Complete the class `SimpleNN` below using `nn.Linear` for the fully connected layers and `nn.ReLU` for activations. Some helpful documentation:

1. `nn.Linear` usage: `nn.Linear(in_features, out_features)` implements a fully connected layer where `in_features` is the size of the input vector and `out_features` is the size of the output vector.
2. `nn.ReLU` usage: `nn.ReLU()` is an activation that introduces non-linearity in the model by applying the ReLU function: $\text{ReLU}(x) = \max(0, x)$.
3. `__init__` vs `forward`: `__init__` defines the network layers as class attributes, whereas `forward` specifies how data flows through the layers (i.e. how to combine/nest the layers together to go from input `x` to output).

<div class="alert alert-info">
Implement the `__init__` and `forward` methods below. (5 points)
</div>

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(SimpleNN, self).__init__()
        raise NotImplementedError()

    def forward(self, x):
        raise NotImplementedError()

Let's print your model. Make sure the in/out dimensions are compatible from layer to layer.

In [ ]:
model = SimpleNN(
    input_size=2, 
    hidden_size1=16, 
    hidden_size2=8, 
    output_size=2
)
print(model)

In [ ]:
"""Test your neural network code here."""
Grader.run_single_test_inline(TestPSet10, "test_07", locals())

<div class="alert alert-info">
Answer the following multiple choice questions (one answer per question, 10 points):
</div>

**Why might increasing the number of hidden layers improve performance?**

- (a) It helps the model capture more complex, non-linear patterns.
- (b) It reduces the risk of overfitting.
- (c) It makes training faster.
- (d) It is always better than adding more neurons to a single layer.

---

**Why don’t we apply an activation function (e.g., ReLU, sigmoid) to the output layer when using `CrossEntropyLoss`?**

- (a) The output layer doesn’t need to be non-linear.
- (b) The `CrossEntropyLoss` function internally applies `softmax`.
- (c) It makes training faster.
- (d) Activations don’t work well with binary classification.


In [ ]:
# Enter your answer for the two above multiple multiple choice questions in the following tuple 
# # as (q1_ans, q2_ans) e.g. ('a', 'a')
mc_answer_set_1 = (None, None)



In [ ]:
"""Test your multiple choice answers here."""
Grader.run_single_test_inline(TestPSet10, "test_08", locals())

### <a name="overfitting"></a> Training with Early Stopping and Weight Regularization (15 points)

In this part, you'll train the neural network you just defined with the dataset we gave you. We provide all of the training code (again, the important thing is understanding the training logic), but you will have to implement the `early_stopping` function and explore weight regularization in the optimization.

One of the (many) beauties of PyTorch is that we don't need to manually define the loss function or optimization algorithm like we did in the logistic regression problem. Instead, we only need these 2 lines of code:

In [ ]:
criterion = nn.CrossEntropyLoss()  # Loss for classification tasks
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

Below we provide the training loop. Implement the `early_stopping` function, which given the current validation loss `val_loss` and the best validation loss so far `best_val_loss`, it determines if the loss hasn't been decreasing for longer than `patience` epochs, and if so, stops training.

<div class="alert alert-info">
Implement the `early_stopping` function below (5 points)
</div>

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=100, patience=10):
    """
    Train a PyTorch model using SGD and apply early stopping.

    Inputs:
    - model: The PyTorch model to train.
    - train_loader: DataLoader for the training set.
    - val_loader: DataLoader for the validation set.
    - criterion: Loss function (e.g., nn.CrossEntropyLoss).
    - optimizer: Optimizer (e.g., torch.optim.SGD).
    - epochs: Number of training epochs.
    - patience: Number of epochs to wait for improvement in validation loss before stopping.

    Outputs:
    - train_losses: List of average training losses per epoch.
    - val_losses: List of average validation losses per epoch.
    """

    # Training parameters
    train_losses, val_losses = [], []
    best_val_loss = float("inf")  # For early stopping
    counter = 0  # Early stopping counter

    # Training loop
    for epoch in range(epochs):
        # Training phase
        model.train() # Sets the model in training phase (lets gradients flow).
        epoch_train_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()           # Reset gradients
            outputs = model(X_batch)        # Forward pass
            loss = criterion(outputs, y_batch)  # Compute loss
            loss.backward()                 # Backward pass (compute gradients)
            optimizer.step()                # Update weights
            epoch_train_loss += loss.item() # Accumulate training loss
        train_losses.append(epoch_train_loss / len(train_loader))

        # Validation phase
        model.eval() # Sets the model in evaluation phase (freezes the network, so gradients don't flow).
        epoch_val_loss = 0
        with torch.no_grad():  # No gradient computation during evaluation
            for X_batch, y_batch in val_loader:
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                epoch_val_loss += loss.item()
        val_losses.append(epoch_val_loss / len(val_loader))

        # Print progress
        if (epoch+1) % 5 == 0 or epoch == 0:
            print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}")

        # Apply early stopping
        best_val_loss, counter, stop_training = early_stopping(val_losses[-1], best_val_loss, counter, patience)
        if val_losses[-1] == best_val_loss:  # Save the model only if val_loss improves
            torch.save(model.state_dict(), "best_model.pth")
        if stop_training:
            break

    return train_losses, val_losses
        
def early_stopping(val_loss, best_val_loss, counter, patience):
    """
    Implements early stopping logic.

    Inputs:
    - val_loss: Current validation loss.
    - best_val_loss: Best validation loss seen so far.
    - counter: Current count of consecutive epochs without improvement.
    - patience: Number of epochs to wait before stopping.

    Outputs:
    - best_val_loss: Updated best validation loss.
    - counter: Updated counter value.
    - stop_training: Boolean flag indicating whether to stop training.
    """
    raise NotImplementedError()

In [ ]:
"""Test your early stopping code here."""
Grader.run_single_test_inline(TestPSet10, "test_09", locals())

Now that we have the training loop, we can evaluate the model and visualize the loss curves.

In [ ]:
# Change these parameters.
decay = 0.001
patience = 10

# Initialize the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=decay)  # Adjust weight_decay as needed

# Train the model
train_losses, val_losses = train_model(
    model, train_loader, val_loader, criterion, optimizer, epochs=100, patience=patience
)

# Plot loss curves
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label="Training Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Loss Curve")
plt.legend()
plt.show()

# Load the best model for evaluation
model.load_state_dict(torch.load("best_model.pth"))
model.eval()

# Evaluate on the test set
correct, total = 0, 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)  # Get class predictions
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

<div class="alert alert-info">
Try varying the early stopping `patience` parameter and the weight regularization parameter `decay`. Then mark which of the following statements are true in the code below.
</div>

#### Multiple choice (10 points)

Select which of the following statements are true (any number of them can be true):

**1. How does the patience parameter in early stopping affect training?**
- (a) Longer patience reduces the chance of stopping too early.
- (b) Shorter patience stops training faster but risks underfitting.
- (c) Patience only affects validation loss, not training.

---

**2. If validation loss fluctuates, how might patience help stabilize early stopping?**
- (a) Patience allows the model to continue training despite small fluctuations in validation loss.
- (b) Patience has no effect if validation loss is fluctuating.
- (c) Fluctuations in validation loss always indicate overfitting.


---

**3. Why does early stopping use validation loss instead of training loss?**
- (a) Validation loss better reflects generalization to unseen data.
- (b) Training loss decreases continuously and doesn’t indicate overfitting.
- (c) Validation loss is typically noisier, making it better for monitoring.

---

**4. What effect does weight decay have on training?**
- (a) It penalizes large weights, reducing overfitting.
- (b) It makes the training loss larger.
- (c) It speeds up convergence.
- (d) It has no effect on validation loss.


---

**5. What might happen if weight decay is set too high or too low?**
- (a) If too high, the model might underfit the data.
- (b) If too low, the model might overfit the training data.
- (c) Weight decay has no significant impact on training or validation.


In [ ]:
# Mark the statements that are true for each of the above questions. 
# E.g. if A in question 1 is true set a=True in the q1 dict.

mc_answer_set_2 = dict(
    q1 = dict(a=False, b=False, c=False),
    q2 = dict(a=False, b=False, c=False),
    q3 = dict(a=False, b=False, c=False),
    q4 = dict(a=False, b=False, c=False, d=False),
    q5 = dict(a=False, b=False, c=False),
)



In [ ]:
"""Test your multiple choice answers here."""
Grader.run_single_test_inline(TestPSet10, "test_10", locals())

# <a name="part4"></a> Time Spent on Pset (5 points)

Please use [this form](https://forms.gle/GJ4MkpMFkxMAPL2D8) to tell us how long you spent on this pset. After you submit the form, the form will give you a confirmation word. Please enter that confirmation word below to get an extra 5 points. 

In [ ]:
form_confirmation_word = "" 


In [ ]:
Grader.grade_output([TestPSet10], [locals()], "results.json")
Grader.print_test_results("results.json")